<a href="https://colab.research.google.com/github/RbsFaneva/ML_exam_2025_ESIIA_5/blob/main/poketrafinday/notebooks/fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')

# Chargement des données
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("=== ANALYSE EXPLORATOIRE ===")
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print("\nPremières lignes du train set:")
print(train_df.head())

In [ ]:
# Distribution de la variable cible
plt.figure(figsize=(10, 6))
fraud_counts = train_df['is_fraud'].value_counts()
plt.pie(fraud_counts, labels=['Non Fraude', 'Fraude'], autopct='%1.1f%%', colors=['lightblue', 'red'])
plt.title('Distribution des Transactions - Fraude vs Non Fraude')
plt.show()

print(f"Nombre de fraudes: {fraud_counts[1]} ({fraud_counts[1]/len(train_df)*100:.2f}%)")
print(f"Déséquilibre important: Ratio de {fraud_counts[0]/fraud_counts[1]:.1f}:1")

In [ ]:
# Analyse des données manquantes
print("\n=== DONNÉES MANQUANTES ===")
missing_data = train_df.isnull().sum()
print(missing_data[missing_data > 0])

# Types de données
print("\n=== TYPES DE DONNÉES ===")
print(train_df.dtypes)

In [ ]:
# Variables catégorielles
categorical_cols = df.select_dtypes(include=['object']).columns
print("\n=== VARIABLES CATÉGORIELLES ===")
for col in categorical_cols:
    if col in train_df.columns:
        print(f"\n{col}:")
        print(train_df[col].value_counts())